<a href="https://colab.research.google.com/github/harjeet88/google_collab/blob/main/solution/lc/sql50/consec_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!chmod -R +x /content/drive/MyDrive/installs/

In [3]:
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']
# Verify installation
!java -version
!which java

openjdk version "11.0.2" 2019-01-15
OpenJDK Runtime Environment 18.9 (build 11.0.2+9)
OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
/content/drive/MyDrive/installs/jdk-11.0.2//bin/java


In [4]:
import pyspark
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ["SPARK_HOME"] = "drive/MyDrive/installs/spark-3.5.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/installs/py_package') # Replace with your actual path

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )

In [ ]:
spark

In [ ]:
#

In [ ]:
# prompt: create and edit manually contents of a csv file to input data

%%writefile /content/drive/MyDrive/data/leetcode_sql180/increment_data.csv
id,num
1,1
2,1
3,1
4,9
5,2
6,1
7,8
8,4
9,6
10,10

Overwriting /content/drive/MyDrive/data/leetcode_sql180/increment_data.csv


In [ ]:
import pandas as pd
import numpy as np

def generate_sample_data(num_rows):
    data = {'id': range(1, num_rows + 1)}

    # Generate random numbers with some duplicates
    num_values = np.random.randint(1, 10, size=num_rows)
    data['num'] = num_values

    df = pd.DataFrame(data)
    return df

# Specify the number of rows
num_rows = 100

# Generate the DataFrame
df = generate_sample_data(num_rows)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/data/leetcode_sql180/increment_data.csv', index=False)

In [8]:
df = spark.read.csv('/content/drive/MyDrive/data/leetcode_sql180/increment_data.csv', header=True, inferSchema=True)
df.show()

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  9|
|  5|  2|
|  6|  1|
|  7|  8|
|  8|  4|
|  9|  6|
| 10| 10|
+---+---+



In [9]:
df1=df
df2=df

In [11]:
#from pyspark.sql.functions import col, distinct
from pyspark.sql.functions import col, countDistinct

In [12]:
joined_df = df.alias("a") \
             .join(df.alias("b"), col("a.id") - 1 == col("b.id"), "inner") \
             .join(df.alias("c"), col("a.id") + 1 == col("c.id"), "inner")

In [13]:
joined_df.show()


+---+---+---+---+---+---+
| id|num| id|num| id|num|
+---+---+---+---+---+---+
|  2|  1|  1|  1|  3|  1|
|  3|  1|  2|  1|  4|  9|
|  4|  9|  3|  1|  5|  2|
|  5|  2|  4|  9|  6|  1|
|  6|  1|  5|  2|  7|  8|
|  7|  8|  6|  1|  8|  4|
|  8|  4|  7|  8|  9|  6|
|  9|  6|  8|  4| 10| 10|
+---+---+---+---+---+---+



In [14]:
# Filter for rows where all three records have the same number
filtered_df = joined_df.where(col("a.num") == col("b.num")) \
                       .where(col("a.num") == col("c.num"))

# Select distinct consecutive numbers
result_df = filtered_df.select(countDistinct(col("a.num")).alias("ConsecutiveNums"))

# Show the result
result_df.show()

+---------------+
|ConsecutiveNums|
+---------------+
|              1|
+---------------+

